In [0]:
# TODO: Tidy the code

In [ ]:
import os
import zipfile
from tqdm import tqdm

In [0]:
os.mkdir(os.path.join('/tmp/', 'data'))
os.mkdir(os.path.join('/tmp/data', 'train'))

In [0]:
# move images from one path to another path


app_train = '/tmp/train/' # current path
new_dir_path = '/tmp/data/train/' # new_dir path
for files in ['denims', 'flip_flops', 'flip_flop_women', 'saree', 'formal_shoes', 'kurta_women']: # Folder names in the current path
  os.mkdir(os.path.join(new_dir_path, files))
  with tqdm(total = len(os.listdir(os.path.join(app_train, files)))) as pbar:
    for file_ in os.listdir(os.path.join(app_train, files)):
      name = file_.split('/')[-1]
      os.rename(os.path.join(app_train, files, name), os.path.join(new_dir_path, files, name))
      pbar.update(1)



100%|██████████| 1316/1316 [00:00<00:00, 31000.34it/s]


In [0]:
train_dir = '/tmp/data/train/'

In [0]:
def move_files(frm_folder, main_dir, child_dir, lst_dir):
  # TO-CHECK
  '''
  frm_folder: Folder in which files are present,
  main_dir: Name and path of directory to which folders must be transferred,
  child_dir: `train` or `test` or `valid` directory,
  lst_dir: list of folders inside the frm_folder to be transferred/moved to the new destination `main_dir/child_dir/lst_dir[i]` where
            `i` can be 1, 2, 3, ... len(lst_dir)
  '''
  
  
  os.mkdir(main_dir)
  os.mkdir(os.path.join(main_dir, child_dir))
  for files in lst_dir:
    os.mkdir(os.path.join(main_dir, child_dir, files))
    with tqdm(total = len(os.listdir(os.path.join(frm_folder, files)))) as pbar:
      for file_ in os.listdir(os.path.join(frm_folder, files)):
        name = file_.split('/')[-1]
        os.rename(os.path.join(frm_folder, files, name), os.path.join(main_dir, child_dir, files, name))
        pbar.update(1)

In [0]:
import os 
import zipfile
import numpy as np
import torch
import torch.nn as nn
import torchvision
import random
import matplotlib.pyplot as plt
from tqdm import tqdm
import torchvision.utils
from PIL import Image
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import torch.nn.functional as F
import pandas as pd
from sklearn.utils import shuffle
from pprint import pprint
from collections import defaultdict

In [0]:
# Get number of files inside sub-folders of a folder
def get_len_directory(folderPath):
  count = 0
  for folder in os.listdir(folderPath):
    for files in os.listdir(os.path.join(folderPath, folder)):
      count = count + 1  

  return count

In [0]:
get_len_directory(train_dir)

2536

In [0]:
path_data_dict = dict()
for folders in os.listdir(train_dir):
  for files in os.listdir(os.path.join(train_dir, folders)):
    path_data_dict[str(os.path.join(train_dir, folders, files))] = [folders]

In [0]:
train_data_frame = pd.DataFrame(path_data_dict, index = None).T
train_data_frame.columns = ['cls']

In [0]:
def get_df_attrs(data_frame_name, list_column_names, have_column_names = True):
  if have_column_names:
    for column in list_column_names:
      return data_frame_name.index.values.tolist(),[r for r in data_frame_name[column]]

In [0]:
train_data_frame = shuffle(train_data_frame)

In [0]:
train_data_frame.replace({'denims': 0, 'flip_flops': 1, 'flip_flop_women': 2, 'saree': 3, 'formal_shoes': 4, 'kurta_women': 5}, inplace = True)

In [0]:
paths, labels = get_df_attrs(train_data_frame, ['cls'])

In [0]:
dict_lbs = defaultdict(list)
tt = train_data_frame.index.values.tolist()
tt_cls = train_data_frame['cls'].values.tolist()


In [0]:
for i in range(len(tt)):
  dict_lbs[tt_cls[i]].append(tt[i])

In [0]:
dict_lbs.keys()

dict_keys([2, 0, 3, 1])

In [0]:
train_data_frame.to_csv('train_data_frame.csv')

In [0]:
class CustomMAMLDatasetGenerator(Dataset):
  

  '''
  Batches and Set. A batch contains sets, and set contains n_way * k_shot for meta-train set, n_way * n_query for meta-test set.
  '''
  
  
  def __init__(self, root, dict_lbs, mode, batchsz, n_way, k_shot, k_query, resize, startidx = 0):
    
    '''
    root: train/valid image directory
    dataframe: dataframe with path and labels
    mode: `train/valid` depending upon the dataset generation
    batchsz: batch size for set, not for images
    n_way
    k_shot
    k_query: number of query images per class
    resize: resize images to
    startidx: can provide any index to start from
    ''' 
    
    self.batchsz = batchsz
    self.dict_lbs = dict_lbs
    self.n_way = n_way
    self.k_shot = k_shot
    self.k_query = k_query
    self.setsz = self.n_way * self.k_shot # number of samples per set
    self.querysz = self.n_way * self.k_query # number of samples per set for evaluation
    self.resize = resize
    self.startidx = startidx
    
    
    pprint("DataSet: {}, Batch Size: {}, {}-Shot, {}-Way, {}-Query, Resize: {}".format(mode, batchsz, k_shot, n_way, k_query, resize))
    
    
    if mode == 'train':
      
      self.transform = transforms.Compose([lambda x: Image.open(x).convert('RGB'),
                                           transforms.Resize((self.resize, self.resize)),
                                           transforms.ToTensor(),
                                           transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
                                          ])
      
    else:
      
      self.transform = transforms.Compose([lambda x: Image.open(x).convert('RGB'),
                                           transforms.Resize((self.resize, self.resize)),
                                           transforms.ToTensor(),
                                           transforms.Normalize((0.485, 0.456, 0.406), 
                                                                (0.229, 0.224, 0.225))
                                          ])
      

    
    self.data = (self.dict_lbs.keys())
    
    print("Data: ".format(self.data))
    
    self.cls_num = len(self.data)
    
#     pprint("Classes: {}".format(self.data))
    
    pprint("Total_classes: {}".format(self.cls_num))
    
#     pprint("Img2Label: {}".format(self.img2label))
    
    self.create_batch(self.batchsz)
    
    
  def create_batch(self, batchsz):
    
    '''
    Creating batches for Meta Learning
    Episode means a batch and how many sets we want to retain
    episodes: batch size
    
    '''
    
    self.support_x_batch = []
    self.query_x_batch = []
    
    self.support_y_batch = []
    
    self.query_y_batch = []
    
    for b in range(self.batchsz):
      
      selected_cls = np.random.choice(self.cls_num, self.n_way, False)
      
#       print("Selected Classes: {}".format(selected_cls))
      
      np.random.shuffle(selected_cls)
      
#       support_x = defaultdict(list)
      
      
      
      support_x = []
      query_x = []
      
      support_y = []
      
      query_y = []  
      
      for cls in selected_cls:
        
#         pprint("Number of images in the selected class: {} are {}".format(cls, [len([c for c in self.label_dict[i]]) for i in self.label_dict.keys()][cls]))
        
#         selected_img_idx = np.random.choice(len([[c for c in self.label_dict[i]] for i in self.label_dict.keys()][cls]), self.k_shot + self.k_query, False)
    
        selected_img_idx = np.random.choice(len([c for c in self.dict_lbs[cls]]), self.k_shot + self.k_query, False)
    
        
      
#         pprint("Selected Image Indexes: {}".format(selected_img_idx))
        
        indexDtrain = np.array(selected_img_idx[:self.k_shot])
        
        indexDtest = np.array(selected_img_idx[self.k_shot:])
        
#         pprint("Train Index: {}".format(indexDtrain))
        
#         pprint("All Classes Train: {}".format([cls]*len(indexDtrain)))
        
#         pprint("Test Index: {}".format(indexDtest))
        
#         pprint("Random: {}".format(len([c for c in self.dict_lbs[cls]])))
        
#         pprint("Selected Images: {}".format([[[c for c in self.dict_lbs[cls]][i]] for i in selected_img_idx]))
        
        support_x.append(np.array([[c for c in self.dict_lbs[cls]][i] for i in indexDtrain]))
        
        query_x.append(np.array([[c for c in self.dict_lbs[cls]][i] for i in indexDtest]))
        
        support_y.append(np.array([cls]*len(indexDtrain)))
        
        query_y.append(np.array([cls]*len(indexDtest)))
        
       
        
      
      temp_support = list(zip(support_x, support_y))
      random.shuffle(temp_support)
      support_x, support_y = zip(*temp_support)
      
      temp_query = list(zip(query_x, query_y))
      random.shuffle(temp_query)
      query_x, query_y = zip(*temp_query)
      
      

#       random.shuffle(support_x)
#       random.shuffle(query_x)
      
#       break
      
      self.support_x_batch.append(support_x)
      self.query_x_batch.append(query_x)
      self.support_y_batch.append(support_y)
      self.query_y_batch.append(query_y)
    
#     flatten_support_x = lambda l: [item for sublist in l for item in sublist]
    
#     fs = flatten_support_x(self.support_x_batch[0])
#     print("Flatten Support: {}".format(fs))
#     print("Len: {}".format(len(fs)))
    
#     pprint("Support X Batch: {}".format(len(self.support_x_batch[0])))
#     pprint("Query X Batch: {}".format(len(self.query_x_batch[0])))
    
  def __getitem__(self, index):
    
    pprint("Index ---------------- : {}".format(index))
    
    support_x = torch.FloatTensor(self.setsz, 3, self.resize, self.resize)
    
    support_y = np.zeros((self.setsz), dtype = np.int)
    
    query_x = torch.FloatTensor(self.querysz, 3, self.resize, self.resize)
    
    query_y = np.zeros((self.querysz), dtype = np.int)
    
    flatten = lambda l: [item for sublist in l for item in sublist]
    
    flatten_support_x = flatten(self.support_x_batch[index])
    
    support_y = flatten(self.support_y_batch[index])
    
    flatten_query_x = flatten(self.query_x_batch[index])
    
    query_y = flatten(self.query_y_batch[index])
    

    unique = np.unique(support_y)
    
    random.shuffle(unique)
    
    support_y_relative = np.zeros(self.setsz)
    
    query_y_relative = np.zeros(self.querysz)
    
    for idx, l in enumerate(unique):
      
      support_y_relative[support_y == l] == idx
      query_y_relative[query_y == l] == idx
      
    for i, path in enumerate(flatten_support_x):
      support_x[i] = self.transform(path)
      
    for i, path in enumerate(flatten_query_x):
      query_x[i] = self.transform(path)
      
    return support_x, torch.LongTensor(support_y_relative), query_x, torch.LongTensor(query_y_relative)
  
  
  def __len__(self):
    
    return self.batchsz

In [0]:
!pip install iteration_utilities

In [0]:
from iteration_utilities import deepflatten

In [0]:
class CustomDatasetGenerator(Dataset):
  
  def __init__(self, root, dictionary, mode, batchsz, n_way, k_shot, k_query, resize, startidx = 0):
    
    self.batchsz = batchsz
    self.dict_lbs = dictionary
    self.n_way = n_way
    self.k_shot = k_shot
    self.k_query = k_query
    self.setsz = self.n_way * self.k_shot
    self.querysz = self.n_way * self.k_query
    self.resize = resize
    self.startidx = startidx
    
    self.transform = transforms.Compose([
        lambda x: Image.open(x).convert("RGB"),
        transforms.Resize((self.resize, self.resize)),
        transforms.ToTensor()
    ])
    
    self.data = (self.dict_lbs.keys())
    pprint("Data Labels: {}".format(self.data))
    
    self.cls_num = len(self.data)
    pprint("Total Classes: {}".format(self.cls_num))
    
    self.create_batch(self.batchsz)
    
  def create_batch(self, batchsz):
    
    support_x_batch = []
    support_y_batch = []
    
    query_x_batch = []
    query_y_batch = []
    
    for b in range(self.batchsz):
      
      
      pprint("Batch Batch {}{}{}".format(b, b, b))
      
      selected_cls = np.random.choice(self.cls_num, self.n_way, False)
      
      pprint("Selected Classes are: {}".format(selected_cls))
      
      np.random.shuffle(selected_cls)
      
      support_x = []
      support_y = []
      
      query_x = []
      query_y = []
      
      for cls in selected_cls:
        
        
        
        pprint("For Class {} -------------".format(cls))
        
        
        
        selected_img_idx = np.random.choice(len(self.dict_lbs[cls]), self.k_shot + self.k_query, False)
        
        pprint("For class {} the images indexes selected are {}".format(cls, selected_img_idx))
        
        indexDtrain = np.array(selected_img_idx[:self.k_shot])
        
        pprint("done Dtrain")
        indexDtest = np.array(selected_img_idx[self.k_shot:])
        
        pprint("done dtest")
        
        
        tmp_s_x = [[c for c in self.dict_lbs[cls]][i] for i in indexDtrain]
        
        pprint("Temperory Support variable: {}, type: {}, Length depth 0: {} ".format(tmp_s_x, type(tmp_s_x), len(tmp_s_x)))
        
        support_x.append(tmp_s_x)
#         support_x.append(np.array([[c for c in self.dict_lbs[cls]][i] for i in indexDtrain]))
        
        tmp_q_x = [[c for c in self.dict_lbs[cls]][i] for i in indexDtest]
    
        pprint("Temporary Query variables: {}, type: {}, Length depth 0: {}".format(tmp_q_x, type(tmp_q_x), len(tmp_q_x)))
        
        query_x.append(tmp_q_x)
        
        pprint("done x")
        support_y.append(np.array([cls]*len(indexDtrain)))
        pprint("done y")
        pprint("Batch: {}, Class: {}, Images in the Support X set: {} ".format(b, cls, len(np.array([[c for c in self.dict_lbs[cls]][i] for i in indexDtrain]))))
        
#         query_x.append(np.array([[c for c in self.dict_lbs[cls]][i] for i in indexDtest]))
        pprint("done q x")
        query_y.append(np.array([cls]*len(indexDtest)))
        pprint("done q y")
        pprint("Batch: {}, Class: {}, Images in the Support Y set: {} ".format(b, cls, len(np.array([[c for c in self.dict_lbs[cls]][i] for i in indexDtest]))))
        
        
        pprint("Size of Support X: {}".format(len(support_x)))
        pprint("Size of Support Y: {}".format(len(support_y)))
        
        pprint("Size of Query X: {}".format(len(query_x)))
        pprint("Size of Query Y: {}".format(len(query_y)))
#       temp_support = list(zip(support_x, support_y))
#       random.shuffle(temp_support)
#       support_x, support_y = zip(*temp_support)
      
#       temp_query = list(zip(query_x, query_y))
#       random.shuffle(temp_query)
#       query_x, query_y = zip(*temp_query)
      
      pprint("Total Number of Images for batch {} in the support set is {}".format(b, len(support_x)))
      pprint("Images in the support set for batch {} are {}".format(b, support_x))
      
    
#     flatten = lambda l: [item for sublist in l for item in sublist]
    
#     support_x = flatten(support_x)
#     query_y = flatten(query_y)
    
      support_x_batch.append(support_x)
      support_y_batch.append(support_y)

      query_x_batch.append(query_x)
      query_y_batch.append(query_y)

      pprint("Size of Size of Support X Batch: {}".format(len(support_x_batch)))
      pprint("Size of Size of Support Y Batch: {}".format(len(support_y_batch)))
      pprint("Size of Size of Query X Batch: {}".format(len(query_x_batch)))
      pprint("Size of Size of Query Y Batch: {}".format(len(query_y_batch)))


      pprint("Total number of images in the support batch {} is {}".format(b, len(support_x_batch)))
      pprint("Total number of images in the query batch {} is {}".format(b, len(query_x_batch)))

      self.support_x_batch = support_x_batch
      self.support_y_batch = support_y_batch

      self.query_x_batch = query_x_batch
      self.query_y_batch = query_y_batch
    
  def __getitem__(self, index):
    
    pprint("Index --------------------- {}".format(index))
    
    support_x = torch.FloatTensor(self.setsz, 3, self.resize, self.resize)
    
    support_y = np.zeros((self.setsz), dtype = np.int)
    
    query_x = torch.FloatTensor(self.querysz, 3, self.resize, self.resize)
    
    query_y = np.zeros((self.querysz), dtype = np.int)
    
    flatten = lambda l: [item for sublist in l for item in sublist]
    
    flatten_support_x = flatten(self.support_x_batch[index])
    
    pprint("Flatten Support X: {}".format(flatten_support_x))
    
    support_y = np.array(flatten(self.support_y_batch[index]))
    
    flatten_query_x = flatten(self.query_x_batch[index])
    
    query_y = np.array(flatten(self.query_y_batch[index]))
    
    pprint("Total Number of items in Support X: {}".format(len(flatten_support_x)))
    pprint("Total Number of items in Query X: {}".format(len(flatten_query_x)))
    
    return flatten_support_x, torch.LongTensor(support_y), flatten_query_x, torch.LongTensor(query_y)
    
  def __len__(self):
    return self.batchsz
        

In [0]:

gen = CustomMAMLDatasetGenerator('/tmp/data/train/', dict_lbs, 'train', n_way = 3, k_shot = 1, k_query = 15, batchsz = 2, resize = 256)

'DataSet: train, Batch Size: 2, 1-Shot, 3-Way, 15-Query, Resize: 256'
Data: 
'Total_classes: 4'


In [0]:
 for i, set_ in enumerate(gen):
  
  X, Y, x, y = set_
  pprint("TYPE: X: {} Y: {} x: {} y: {}".format((X.size()), len(Y), (x.size()), len(y)))

'Index ---------------- : 0'
('TYPE: X: torch.Size([3, 3, 256, 256]) Y: 3 x: torch.Size([45, 3, 256, 256]) '
 'y: 45')
'Index ---------------- : 1'
('TYPE: X: torch.Size([3, 3, 256, 256]) Y: 3 x: torch.Size([45, 3, 256, 256]) '
 'y: 45')
'Index ---------------- : 2'


In [0]:
from torchvision.utils import make_grid

In [0]:
from torchsummary import summary
from collections import OrderedDict


In [0]:
# params
epoch = 60000
n_way = 3
k_spt = 1
k_qry = 15
imgsz = 256
imgc = 3
task_num = 4
meta_lr = 1e-3
update_lr = 0.01
update_step = 5
update_step_test = 10

In [0]:
# Network Configurations

config = [
        ('conv2d', [32, 3, 3, 3, 1, 0]),
        ('relu', [True]),
        ('bn', [32]),
        ('max_pool2d', [2, 2, 0]),
        ('conv2d', [32, 32, 3, 3, 1, 0]),
        ('relu', [True]),
        ('bn', [32]),
        ('max_pool2d', [2, 2, 0]),
        ('conv2d', [32, 32, 3, 3, 1, 0]),
        ('relu', [True]),
        ('bn', [32]),
        ('max_pool2d', [2, 2, 0]),
        ('conv2d', [32, 32, 3, 3, 1, 0]),
        ('relu', [True]),
        ('bn', [32]),
        ('max_pool2d', [2, 1, 0]),
        ('flatten', []),
        ('linear', [n_way, 23328])
    ]


net_config = [              #ip_sz: 3, 256, 256
    ('conv2d', [3, 32, 3]), #op_sz: 32, 254, 254
    ('relu', [True]),
    ('max_pool2d', [2, 2]), #op_sz: 32, 127, 127
    ('conv2d', [32, 64, 4]), #op_sz: 64, 124, 124 
    ('relu', [True]),
    ('max_pool2d', [2, 2]),  #op_sz: 64, 62, 62
    ('conv2d', [64, 128, 3]), #op_sz: 128, 60, 60
    ('relu', [True]),
    ('max_pool2d', [2, 2]),  #op_sz: 128, 30, 30
    ('conv2d', [128, 256, 3]), # op_sz: 256, 28, 28
    ('relu', [True]),        
    ('max_pool2d', [2, 2]),    # op_sz: 256, 14, 14
    ('flatten', []),
    ('linear', [n_way, 23328 ])
]

In [0]:
class Learner(nn.Module):
    """
    """

    def __init__(self, config, imgc, imgsz):
        """
        :param config: network config file, type:list of (string, list)
        :param imgc: 1 or 3
        :param imgsz:  28 or 84
        """
        super(Learner, self).__init__()


        self.config = config

        # this dict contains all tensors needed to be optimized
        self.vars = nn.ParameterList()
        # running_mean and running_var
        self.vars_bn = nn.ParameterList()

        for i, (name, param) in enumerate(self.config):
            if name is 'conv2d':
                # [ch_out, ch_in, kernelsz, kernelsz]
                w = nn.Parameter(torch.ones(*param[:4]))
                # gain=1 according to cbfin's implementation
                torch.nn.init.kaiming_normal_(w)
                self.vars.append(w)
                # [ch_out]
                self.vars.append(nn.Parameter(torch.zeros(param[0])))

            elif name is 'convt2d':
                # [ch_in, ch_out, kernelsz, kernelsz, stride, padding]
                w = nn.Parameter(torch.ones(*param[:4]))
                # gain=1 according to cbfin's implementation
                torch.nn.init.kaiming_normal_(w)
                self.vars.append(w)
                # [ch_in, ch_out]
                self.vars.append(nn.Parameter(torch.zeros(param[1])))

            elif name is 'linear':
                # [ch_out, ch_in]
                w = nn.Parameter(torch.ones(*param))
                # gain=1 according to cbfinn's implementation
                torch.nn.init.kaiming_normal_(w)
                self.vars.append(w)
                # [ch_out]
                self.vars.append(nn.Parameter(torch.zeros(param[0])))

            elif name is 'bn':
                # [ch_out]
                w = nn.Parameter(torch.ones(param[0]))
                self.vars.append(w)
                # [ch_out]
                self.vars.append(nn.Parameter(torch.zeros(param[0])))

                # must set requires_grad=False
                running_mean = nn.Parameter(torch.zeros(param[0]), requires_grad=False)
                running_var = nn.Parameter(torch.ones(param[0]), requires_grad=False)
                self.vars_bn.extend([running_mean, running_var])


            elif name in ['tanh', 'relu', 'upsample', 'avg_pool2d', 'max_pool2d',
                          'flatten', 'reshape', 'leakyrelu', 'sigmoid']:
                continue
            else:
                raise NotImplementedError






    def extra_repr(self):
        info = ''

        for name, param in self.config:
            if name is 'conv2d':
                tmp = 'conv2d:(ch_in:%d, ch_out:%d, k:%dx%d, stride:%d, padding:%d)'\
                      %(param[1], param[0], param[2], param[3], param[4], param[5],)
                info += tmp + '\n'

            elif name is 'convt2d':
                tmp = 'convTranspose2d:(ch_in:%d, ch_out:%d, k:%dx%d, stride:%d, padding:%d)'\
                      %(param[0], param[1], param[2], param[3], param[4], param[5],)
                info += tmp + '\n'

            elif name is 'linear':
                tmp = 'linear:(in:%d, out:%d)'%(param[1], param[0])
                info += tmp + '\n'

            elif name is 'leakyrelu':
                tmp = 'leakyrelu:(slope:%f)'%(param[0])
                info += tmp + '\n'


            elif name is 'avg_pool2d':
                tmp = 'avg_pool2d:(k:%d, stride:%d, padding:%d)'%(param[0], param[1], param[2])
                info += tmp + '\n'
            elif name is 'max_pool2d':
                tmp = 'max_pool2d:(k:%d, stride:%d, padding:%d)'%(param[0], param[1], param[2])
                info += tmp + '\n'
            elif name in ['flatten', 'tanh', 'relu', 'upsample', 'reshape', 'sigmoid', 'use_logits', 'bn']:
                tmp = name + ':' + str(tuple(param))
                info += tmp + '\n'
            else:
                raise NotImplementedError

        return info



    def forward(self, x, vars=None, bn_training=True):
        """
        This function can be called by finetunning, however, in finetunning, we dont wish to update
        running_mean/running_var. Thought weights/bias of bn is updated, it has been separated by fast_weights.
        Indeed, to not update running_mean/running_var, we need set update_bn_statistics=False
        but weight/bias will be updated and not dirty initial theta parameters via fast_weiths.
        :param x: [b, 1, 28, 28]
        :param vars:
        :param bn_training: set False to not update
        :return: x, loss, likelihood, kld
        """

        if vars is None:
            vars = self.vars

        idx = 0
        bn_idx = 0

        for name, param in self.config:
            if name is 'conv2d':
                w, b = vars[idx], vars[idx + 1]
                pprint("Conv2D -- Weight: {}, Bias: {}".format(w.size(), b.size()))
                # remember to keep synchrozied of forward_encoder and forward_decoder!
                x = F.conv2d(x, w, b, stride=param[4], padding=param[5])
                idx += 2
                # print(name, param, '\tout:', x.shape)
            elif name is 'convt2d':
                w, b = vars[idx], vars[idx + 1]
                # remember to keep synchrozied of forward_encoder and forward_decoder!
                x = F.conv_transpose2d(x, w, b, stride=param[4], padding=param[5])
                idx += 2
                # print(name, param, '\tout:', x.shape)
            elif name is 'linear':
                w, b = vars[idx], vars[idx + 1]
                pprint("Linear -- Weight: {}, Bias: {}".format(w.size(), b.size()))
                x = F.linear(x, w, b)
                idx += 2
                # print('forward:', idx, x.norm().item())
            elif name is 'bn':
                w, b = vars[idx], vars[idx + 1]
                running_mean, running_var = self.vars_bn[bn_idx], self.vars_bn[bn_idx+1]
                x = F.batch_norm(x, running_mean, running_var, weight=w, bias=b, training=bn_training)
                idx += 2
                bn_idx += 2

            elif name is 'flatten':
                # print(x.shape)
                x = x.view(x.size(0), -1)
            elif name is 'reshape':
                # [b, 8] => [b, 2, 2, 2]
                x = x.view(x.size(0), *param)
            elif name is 'relu':
                x = F.relu(x, inplace=param[0])
            elif name is 'leakyrelu':
                x = F.leaky_relu(x, negative_slope=param[0], inplace=param[1])
            elif name is 'tanh':
                x = F.tanh(x)
            elif name is 'sigmoid':
                x = torch.sigmoid(x)
            elif name is 'upsample':
                x = F.upsample_nearest(x, scale_factor=param[0])
            elif name is 'max_pool2d':
                x = F.max_pool2d(x, param[0], param[1], param[2])
            elif name is 'avg_pool2d':
                x = F.avg_pool2d(x, param[0], param[1], param[2])

            else:
                raise NotImplementedError

        # make sure variable is used properly
        assert idx == len(vars)
        assert bn_idx == len(self.vars_bn)


        return x


    def zero_grad(self, vars=None):
        """
        :param vars:
        :return:
        """
        with torch.no_grad():
            if vars is None:
                for p in self.vars:
                    if p.grad is not None:
                        p.grad.zero_()
            else:
                for p in vars:
                    if p.grad is not None:
                        p.grad.zero_()

    def parameters(self):
        """
        override this function since initial parameters will return with a generator.
        :return:
        """
        return self.vars

In [0]:
class Learner(nn.Module):
  
  def __init__(self, config, imgc, imgsz):
    
    super(Learner, self).__init__()
    
    self.config = config
    
    # Dictionary containing all the tensors needed to be optimized
    self.vars = nn.ParameterList()
    
    # running mean and running variance
    
    self.vars_bn = nn.ParameterList()
    
    for i, (name, param) in enumerate(self.config):
      
      if name is 'conv2d':
        
        w = nn.Parameter(torch.ones())

In [0]:
class Meta(nn.Module):
    """
    Meta Learner
    """
    def __init__(self, config):
        """
        :param args:
        """
        super(Meta, self).__init__()

        self.update_lr = update_lr
        self.meta_lr = meta_lr
        self.n_way = n_way
        self.k_spt = k_spt
        self.k_qry = k_qry
        self.task_num = task_num
        self.update_step = update_step
        self.update_step_test = update_step_test


        self.net = Learner(config, imgc, imgsz)
        self.meta_optim = torch.optim.Adam(self.net.parameters(), lr=self.meta_lr)




    def clip_grad_by_norm_(self, grad, max_norm):
        """
        in-place gradient clipping.
        :param grad: list of gradients
        :param max_norm: maximum norm allowable
        :return:
        """

        total_norm = 0
        counter = 0
        for g in grad:
            param_norm = g.data.norm(2)
            total_norm += param_norm.item() ** 2
            counter += 1
        total_norm = total_norm ** (1. / 2)

        clip_coef = max_norm / (total_norm + 1e-6)
        if clip_coef < 1:
            for g in grad:
                g.data.mul_(clip_coef)

        return total_norm/counter


    def forward(self, x_spt, y_spt, x_qry, y_qry):
        """
        :param x_spt:   [b, setsz, c_, h, w]
        :param y_spt:   [b, setsz]
        :param x_qry:   [b, querysz, c_, h, w]
        :param y_qry:   [b, querysz]
        :return:
        """
        task_num, setsz, c_, h, w = x_spt.size()
        querysz = x_qry.size(1)

        losses_q = [0 for _ in range(self.update_step + 1)]  # losses_q[i] is the loss on step i
        corrects = [0 for _ in range(self.update_step + 1)]


        for i in range(task_num):

            # 1. run the i-th task and compute loss for k=0
            logits = self.net(x_spt[i], vars=None, bn_training=True)
            loss = F.cross_entropy(logits, y_spt[i])
            grad = torch.autograd.grad(loss, self.net.parameters())
            fast_weights = list(map(lambda p: p[1] - self.update_lr * p[0], zip(grad, self.net.parameters())))

            # this is the loss and accuracy before first update
            with torch.no_grad():
                # [setsz, nway]
                logits_q = self.net(x_qry[i], self.net.parameters(), bn_training=True)
                loss_q = F.cross_entropy(logits_q, y_qry[i])
                losses_q[0] += loss_q

                pred_q = F.softmax(logits_q, dim=1).argmax(dim=1)
                correct = torch.eq(pred_q, y_qry[i]).sum().item()
                corrects[0] = corrects[0] + correct

            # this is the loss and accuracy after the first update
            with torch.no_grad():
                # [setsz, nway]
                logits_q = self.net(x_qry[i], fast_weights, bn_training=True)
                loss_q = F.cross_entropy(logits_q, y_qry[i])
                losses_q[1] += loss_q
                # [setsz]
                pred_q = F.softmax(logits_q, dim=1).argmax(dim=1)
                correct = torch.eq(pred_q, y_qry[i]).sum().item()
                corrects[1] = corrects[1] + correct

            for k in range(1, self.update_step):
                # 1. run the i-th task and compute loss for k=1~K-1
                logits = self.net(x_spt[i], fast_weights, bn_training=True)
                loss = F.cross_entropy(logits, y_spt[i])
                # 2. compute grad on theta_pi
                grad = torch.autograd.grad(loss, fast_weights)
                # 3. theta_pi = theta_pi - train_lr * grad
                fast_weights = list(map(lambda p: p[1] - self.update_lr * p[0], zip(grad, fast_weights)))

                logits_q = self.net(x_qry[i], fast_weights, bn_training=True)
                # loss_q will be overwritten and just keep the loss_q on last update step.
                loss_q = F.cross_entropy(logits_q, y_qry[i])
                losses_q[k + 1] += loss_q

                with torch.no_grad():
                    pred_q = F.softmax(logits_q, dim=1).argmax(dim=1)
                    correct = torch.eq(pred_q, y_qry[i]).sum().item()  # convert to numpy
                    corrects[k + 1] = corrects[k + 1] + correct



        # end of all tasks
        # sum over all losses on query set across all tasks
        loss_q = losses_q[-1] / task_num

        # optimize theta parameters
        self.meta_optim.zero_grad()
        loss_q.backward()
        # print('meta update')
        # for p in self.net.parameters()[:5]:
        # 	print(torch.norm(p).item())
        self.meta_optim.step()


        accs = np.array(corrects) / (querysz * task_num)

        return accs


    def finetunning(self, x_spt, y_spt, x_qry, y_qry):
        """
        :param x_spt:   [setsz, c_, h, w]
        :param y_spt:   [setsz]
        :param x_qry:   [querysz, c_, h, w]
        :param y_qry:   [querysz]
        :return:
        """
        assert len(x_spt.shape) == 4

        querysz = x_qry.size(0)

        corrects = [0 for _ in range(self.update_step_test + 1)]

        # in order to not ruin the state of running_mean/variance and bn_weight/bias
        # we finetunning on the copied model instead of self.net
        net = deepcopy(self.net)

        # 1. run the i-th task and compute loss for k=0
        logits = net(x_spt)
        loss = F.cross_entropy(logits, y_spt)
        grad = torch.autograd.grad(loss, net.parameters())
        fast_weights = list(map(lambda p: p[1] - self.update_lr * p[0], zip(grad, net.parameters())))

        # this is the loss and accuracy before first update
        with torch.no_grad():
            # [setsz, nway]
            logits_q = net(x_qry, net.parameters(), bn_training=True)
            # [setsz]
            pred_q = F.softmax(logits_q, dim=1).argmax(dim=1)
            # scalar
            correct = torch.eq(pred_q, y_qry).sum().item()
            corrects[0] = corrects[0] + correct

        # this is the loss and accuracy after the first update
        with torch.no_grad():
            # [setsz, nway]
            logits_q = net(x_qry, fast_weights, bn_training=True)
            # [setsz]
            pred_q = F.softmax(logits_q, dim=1).argmax(dim=1)
            # scalar
            correct = torch.eq(pred_q, y_qry).sum().item()
            corrects[1] = corrects[1] + correct

        for k in range(1, self.update_step_test):
            # 1. run the i-th task and compute loss for k=1~K-1
            logits = net(x_spt, fast_weights, bn_training=True)
            loss = F.cross_entropy(logits, y_spt)
            # 2. compute grad on theta_pi
            grad = torch.autograd.grad(loss, fast_weights)
            # 3. theta_pi = theta_pi - train_lr * grad
            fast_weights = list(map(lambda p: p[1] - self.update_lr * p[0], zip(grad, fast_weights)))

            logits_q = net(x_qry, fast_weights, bn_training=True)
            # loss_q will be overwritten and just keep the loss_q on last update step.
            loss_q = F.cross_entropy(logits_q, y_qry)

            with torch.no_grad():
                pred_q = F.softmax(logits_q, dim=1).argmax(dim=1)
                correct = torch.eq(pred_q, y_qry).sum().item()  # convert to numpy
                corrects[k + 1] = corrects[k + 1] + correct


        del net

        accs = np.array(corrects) / querysz

        return accs

In [0]:
# net = OmniglotNet(4).cuda()

In [0]:
# summary(net, (3, 256, 256))

In [0]:
maml = Meta(config).cuda()

In [0]:
# train

for e in range(10):  
  db = DataLoader(gen, task_num, shuffle = True, num_workers = 1, pin_memory = True)
  
  
  
  for step, (x_spt, y_spt, x_qry, y_qry) in enumerate(db):
    
    pprint("Step No. : {}".format(step))
    
    pprint("Size X Support: {}, Size X Query: {}, Size Y Support: {}, Size Y Query: {}".format(x_spt.size(), x_qry.size(), y_spt.size(), y_qry.size()))
    
    x_spt, y_spt, x_qry, y_qry = x_spt.cuda(), y_spt.cuda(), x_qry.cuda(), y_qry.cuda()
    
    accs = maml(x_spt, y_spt, x_qry, y_qry)
    
    print("Training Acc: {}".format(accs))

'Index ---------------- : 1'
'Index ---------------- : 0'
'Step No. : 0'
('Size X Support: torch.Size([2, 3, 3, 256, 256]), Size X Query: '
 'torch.Size([2, 45, 3, 256, 256]), Size Y Support: torch.Size([2, 3]), Size Y '
 'Query: torch.Size([2, 45])')
'Conv2D -- Weight: torch.Size([32, 3, 3, 3]), Bias: torch.Size([32])'
'Conv2D -- Weight: torch.Size([32, 32, 3, 3]), Bias: torch.Size([32])'
'Conv2D -- Weight: torch.Size([32, 32, 3, 3]), Bias: torch.Size([32])'
'Conv2D -- Weight: torch.Size([32, 32, 3, 3]), Bias: torch.Size([32])'
'Linear -- Weight: torch.Size([3, 23328]), Bias: torch.Size([3])'
'Conv2D -- Weight: torch.Size([32, 3, 3, 3]), Bias: torch.Size([32])'
'Conv2D -- Weight: torch.Size([32, 32, 3, 3]), Bias: torch.Size([32])'
'Conv2D -- Weight: torch.Size([32, 32, 3, 3]), Bias: torch.Size([32])'
'Conv2D -- Weight: torch.Size([32, 32, 3, 3]), Bias: torch.Size([32])'
'Linear -- Weight: torch.Size([3, 23328]), Bias: torch.Size([3])'
'Conv2D -- Weight: torch.Size([32, 3, 3, 3]), Bias